In [1]:
import torch
from virtuoso import model as modelzoo
from virtuoso import model_parameters as param
from omegaconf import OmegaConf
import _pickle as pickle
import sys
import yaml
from virtuoso import parser

sys.modules['model_parameters'] = param


In [4]:
args = parser.get_parser()
args

ArgumentParser(prog='virtuosonet', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

ArgumentParser(prog='virtuosonet', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

In [15]:
conf = OmegaConf.create(dict)
with open('isgn_param.yml', 'w') as f:
    yaml.dump(dict, f, default_flow_style=False)

In [79]:
model = modelzoo.ISGN(conf, 'cpu')
model

ISGN(
  (note_fc): Sequential(
    (0): Linear(in_features=78, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Dropout(p=0.1, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): ReLU()
  )
  (graph_1st): GatedGraph(
    (sigmoid): Sigmoid()
    (tanh): Tanh()
  )
  (graph_between): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
  )
  (graph_2nd): GatedGraph(
    (sigmoid): Sigmoid()
    (tanh): Tanh()
  )
  (measure_attention): ContextAttention(
    (attention_net): Linear(in_features=256, out_features=256, bias=True)
  )
  (measure_rnn): LSTM(256, 64, num_layers=2, batch_first=True, bidirectional=True)
  (performance_contractor): Sequential(
    (0): Linear(in_features=523, out_features=64, bias=True)
    (1): D

In [74]:
with open('isgn_param.yml', 'r') as f:
    yaml_obj = yaml.load(f, Loader=yaml.FullLoader)

In [77]:
conf

{'beat': {'input': 0, 'layer': 1, 'margin': 0, 'size': 0}, 'encoded_vector_size': 16, 'encoder': {'input': 523, 'layer': 2, 'margin': 0, 'size': 64}, 'final': {'input': 512, 'layer': 1, 'margin': 32, 'size': 0}, 'graph_iteration': 4, 'hierarchy_level': None, 'input_size': 78, 'is_baseline': False, 'is_graph': False, 'is_simplified': False, 'is_teacher_force': False, 'is_test_version': False, 'margin': {'input': 0, 'layer': 1, 'margin': 0, 'size': 0}, 'measure': {'input': 0, 'layer': 2, 'margin': 0, 'size': 64}, 'note': {'input': 0, 'layer': 2, 'margin': 0, 'size': 128}, 'num_attention_head': 8, 'num_edge_types': 10, 'onset': {'input': 0, 'layer': 1, 'margin': 0, 'size': 0}, 'output_size': 11, 'section': {'input': 0, 'layer': 1, 'margin': 0, 'size': 0}, 'sequence_iteration': 3, 'sum': {'input': 0, 'layer': 1, 'margin': 0, 'size': 0}, 'time_reg': {'input': 0, 'layer': 2, 'margin': 0, 'size': 32}, 'training_args': {'boolPedal': False, 'composer': 'Beethoven', 'dataName': 'pedal_refresh', 

In [6]:
with open("isgn_param.dat", 'rb') as f:
    param = pickle.load(f)
with open("prime_isgn_best.pth.tar", 'rb') as f:
    weights = torch.load(f)

In [13]:
dict = {'nn_params': {}, 'training_params':{}}
for key in vars(param):
    if isinstance(getattr(param,key), param.Param):
        dict['nn_params'][key] = {}
        for subkey in vars(getattr(param, key)):
            dict['nn_params'][key][subkey] = getattr(getattr(param, key), subkey)
    elif key == 'training_args':
        dict['training_params'][key] = {}
        for subkey in vars(param.training_args):
            dict['training_params'][key][subkey] = getattr(param.training_args, subkey)
    else:
        dict['nn_params'][key] = getattr(param, key)

dict


{'nn_params': {'note': {'size': 128, 'layer': 2, 'input': 0, 'margin': 0},
  'onset': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'beat': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'measure': {'size': 64, 'layer': 2, 'input': 0, 'margin': 0},
  'section': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'final': {'size': 0, 'layer': 1, 'input': 512, 'margin': 32},
  'voice': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'sum': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'encoder': {'size': 64, 'layer': 2, 'input': 523, 'margin': 0},
  'time_reg': {'size': 32, 'layer': 2, 'input': 0, 'margin': 0},
  'margin': {'size': 0, 'layer': 1, 'input': 0, 'margin': 0},
  'input_size': 78,
  'output_size': 11,
  'encoded_vector_size': 16,
  'graph_iteration': 4,
  'sequence_iteration': 3,
  'num_edge_types': 10,
  'num_attention_head': 8,
  'is_graph': False,
  'is_teacher_force': False,
  'is_baseline': False,
  'hierarchy_level': None,
  'is_simplified': False,
 

In [44]:
test = dict['training_args']
for 

False

In [ ]:


if 'isgn' in args.modelCode:
    MODEL = modelzoo.ISGN(NET_PARAM, device).to(device)
elif 'han' in args.modelCode:
    if 'ar' in args.modelCode:
        step_by_step = True
    else:
        step_by_step = False
    MODEL = modelzoo.HAN_Integrated(NET_PARAM, device, step_by_step).to(device)
elif 'trill' in args.modelCode:
    MODEL = modelzoo.TrillRNN(NET_PARAM, device).to(device)
else:
    print('Error: Unclassified model code')
    # Model = modelzoo.HAN_VAE(NET_PARAM, device, False).to(device)